# H8 Comparing Methods

We've seen two different ways to simulate physical systems in this module so far: Trotterization, and linear combination of unitaries (LCU). Which should you use? It turns out that each method works better in certain situations. Both are useful tools for the toolbox of the quantum physicist! To understand when to use LCU and when to use Trotterization, we'll look at how the simulation cost for a given error depends on the Hamiltonian at hand.

Author: [Monit Sharma](https://github.com/MonitSharma)
LinkedIn: [Monit Sharma](https://www.linkedin.com/in/monitsharma/)
Twitter: [@MonitSharma1729](https://twitter.com/MonitSharma1729)
Medium : [MonitSharma](https://medium.com/@_monitsharma)

### Codercise H.8.1. 
(a) Complete the code for trotter_XandZ below to Trotterize evolution with the Hamiltonian . The exact result exact_result_XandZ is provided for you. Submitting will generate a plot of the error trotter_error_XandZ against the number of steps on a log-log scale for two different choices of  and .



In [ ]:
dev = qml.device("default.qubit", wires=1)

def exact_result_XandZ(alpha, beta, time):
    """Exact circuit for evolving a qubit with H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        
    Returns: 
        array[complex]: The exact state after evolution.
    """
    root = np.sqrt(alpha**2 + beta**2)
    c_0 = np.cos(root*time) - (alpha/root)*np.sin(root*time)*1.j
    c_1 = -(beta/root)*np.sin(root*time)*1.j
    return np.array([c_0, c_1])
    
@qml.qnode(dev)
def trotter_XandZ(alpha, beta, time, n):
    """Trotterized circuit for evolving a qubit with H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        array[complex]: The state after applying the Trotterized circuit.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    t = time
    for _ in range(n):
	    qml.PauliRot(2 * t/n * alpha, "Z",wires=0)
	    qml.PauliRot(2 * t/n * beta, "X",wires=0)

    return qml.state()

def trotter_error_XandZ(alpha, beta, time, n):
    """Difference between the exact and Trotterized result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        float: The distance between the exact and Trotterized result.
    """
    diff = np.abs(trotter_XandZ(alpha, beta, time, n) - exact_result_XandZ(alpha, beta, time))
    return np.sqrt(sum(diff*diff))


###Codercise H.8.1. 
(b) Implement the second-order Trotterization circuit trotter_2_XandZ. The error for the first-order and second-order Trotter methods are shown as a function of step number for the same Hamiltonian () on a log-log plot. Are the results what you expect?

In [ ]:
@qml.qnode(dev)
def trotter_2_XandZ(alpha, beta, time, n):
    """Second-order Trotter circuit for the Hamiltonian H = alpha Z +  beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        array[complex]: The state after applying the second-order circuit.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    t = time
    for _ in range(n):
	    qml.PauliRot(2 * t/(2*n) * alpha, "Z", wires=0)
	    qml.PauliRot(2 * t/(2*n) * beta, "X", wires=0)
	    qml.PauliRot(2 * t/(2*n) * beta, "X", wires=0)
	    qml.PauliRot(2 * t/(2*n) * alpha, "Z", wires=0)


    return qml.state()

def trotter_2_error_XandZ(alpha, beta, time, n):
    """Difference between the exact and second-order Trotter result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        float: The distance between the exact and second-order result.
    """
    diff = np.abs(trotter_2_XandZ(alpha, beta, time, n) - exact_result_XandZ(alpha, beta, time))
    return np.sqrt(sum(diff*diff))


### Codercise H.8.1. 
(c) Finish the code to implement order- Trotterization below, trotter_k_XandZ. This implements the unitary  as a subcircuit. We use this function to plot error as a function of step number for different orders of Trotterization. Are the results as expected?

Tip. Note that it is conventional to refer to the order of the Trotterization as , the power at which the error falls off.

In [ ]:
@qml.qnode(dev)
def trotter_k_XandZ(alpha, beta, time, n, k):
    """
    Order-2k Trotter circuit for the Hamiltonian H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        k (int): The order of our Trotterization formula divided by 2.
        
    Returns: 
        array[complex]: The state after applying the order-2k circuit.
    """
    def U(alpha, beta, time, n, k):
        if k == 1:
            qml.RZ(alpha*time/n, wires=[0])
            qml.RX(2*beta*time/n, wires=[0])
            qml.RZ(alpha*time/n, wires=[0])
        else:
            ##################
            # YOUR CODE HERE #
            ##################
            s = 1 / (4 - 4 ** (1 / (2 * k -1)))
            U(alpha, beta, s * time, n, k -1)
            U(alpha, beta, s * time, n, k -1)
            U(alpha, beta, (1 - 4 * s) * time, n , k-1)
            U(alpha, beta, s * time, n ,k-1)
            U(alpha, beta, s * time, n, k -1)

            pass # MODIFY THIS
    for _ in range(n):
        U(alpha, beta, time, n, k)
    return qml.state()

def trotter_k_error_XandZ(alpha, beta, time, n, k):
    """
    Difference between the exact and order-2k Trotter result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        k (int): The order of our Trotterization formula divided by 2.
        
    Returns: 
        float: The distance between the exact and order-2k result.
    """
    diff = np.abs(trotter_k_XandZ(alpha, beta, time, n, k) - exact_result_XandZ(alpha, beta, time))
    return np.sqrt(sum(diff*diff))


### Codercise H.8.1. 
(d) Finish the function below to determine the number of gates  needed to achieve a given accuracy  in the order- Trotterized time evolution. Note that trotter_k_error_XandZ(alpha, beta, time, n, k) is defined for you. Submitting produces a plot of the number of steps as a function of order , for a specific error . In this case, what is the optimal order, and how many gates are needed to achieve it?

In [ ]:
def trotter_steps_XandZ(alpha, beta, time, error, k):
    """
    Computes the number of Trotter steps needed for a given order k and error.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        error (float): The size of the tolerated simulation error.
        k (int): The order of our Trotterization formula divided by 2.
        
    Returns: 
        int: The number of steps needed to achieve a given error.
    """
    n = 1
    while True:
	    e = trotter_k_error_XandZ(alpha, beta, time,n,k)
	    if e <= error:
		    break
	    n+= 1
    ##################
    # YOUR CODE HERE #
    ##################
    return n

error = 1e-6
optimal_k = 3 # MODIFY THIS AFTER LOOKING AT THE PLOT 
n = trotter_steps_XandZ(1, 1, 1, error, optimal_k)
depth = qml.specs(trotter_k_XandZ)(1, 1, 1, n, optimal_k)['depth']
print("The Trotter circuit of order", 2*optimal_k, "uses a circuit of depth", depth, "gates to achieve error ε =", error, ".")


### Codercise H.8.2. 
(a) Complete the function below to generate a list of (positive) coefficients and unitaries for truncating the Taylor series to  terms. The factorial function is provided to you as fact(n).

In [ ]:
def truncation_XandZ(alpha, beta, time, K_bits):
    """
    Generates unitaries and coefficients for the truncated X and Z evolution.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        K_bits (int): The index of the truncation order, K = 2^K_bits.
        
    Returns: 
        [array[complex], array[array(complex)]]: Coefficients and unitaries.
    """
    root = np.sqrt(alpha**2 + beta**2)
    coeff_list = [0]*2**K_bits
    U_list = [0]*2**K_bits
    V = (alpha*qml.PauliZ(wires=0).compute_matrix() + beta*qml.PauliX(wires=0).compute_matrix())/root

    for k in range(2**(K_bits-1)):
        
        coeff_list[2*k] = ((time * np.sqrt(alpha**2 + beta**2))**(2*k))/(fact(2*k)) # MODIFY THIS
        coeff_list[2*k + 1] = ((time * np.sqrt(alpha**2 + beta**2))**(2*k + 1)) / (fact(2 * k +1)) # MODIFY THIS
        U_list[2*k] = np.eye(2) * (-1)**k # MODIFY THIS
        U_list[2*k + 1] = (-1)**k * (-1j) * V # MODIFY THIS

    return [coeff_list, U_list]


### Codercise H.8.2. 
(b) Complete the circuit for performing LCU simulation. Recall that it takes the following form:

![](https://codebook.xanadu.ai/pics/ps-oracles.svg)

In [ ]:
def LCU_XandZ(alpha, beta, time, K_bits):
    """
    LCU circuit for simulating the Hamiltonian H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        K_bits (int): The index of the truncation order, K = 2^K_bits.
        
    Returns: 
        array[complex]: The state after applying the LCU circuit.
    """
    aux = range(K_bits) # The auxiliary qubits
    main = range(K_bits, K_bits + 1) # The main system
    dev2 = qml.device("default.qubit", wires=K_bits + 1, shots=None)
    [coeff_list, U_list] = truncation_XandZ(alpha, beta, time, K_bits)
    
    @qml.qnode(dev2)
    def LCU_circuit():

        ##################
        # YOUR CODE HERE #
        ##################
        print(PREPARE_matrix(coeff_list))
        qml.QubitUnitary(PREPARE_matrix(coeff_list),wires=aux)
        SELECT(U_list)
        qml.QubitUnitary(PREPARE_matrix(coeff_list).conj().T, wires=aux)


        return qml.state()

    unnormed = LCU_circuit()[:2] # Unnormalized state of main qubit
    normed = unnormed/np.sqrt(sum(np.conjugate(unnormed)*unnormed)) # Normalize!
    
    return normed


### Codercise H.8.2. 
(c) As we did for Trotterization, let's compute the error  as a function of the truncation order, and generate the accompanying plot. Recall that the exact result is given by the function exact_result_XandZ(alpha, beta, time) and the LCU circuit approximation by LCU_XandZ(alpha, beta, time, K_bits). How do your results compare to Trotterization?

In [ ]:
def LCU_error_XandZ(alpha, beta, time, K_bits):
    """
    Difference between the exact and LCU simulation result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        K_bits (int): The index of the truncation order, K = 2^K_bits.
        
    Returns: 
        float: The distance between the exact and LCU result.
    """

    diff = np.abs(LCU_XandZ(alpha, beta, time, K_bits) - exact_result_XandZ(alpha, beta,time)) # MODIFY THIS

    return np.sqrt(sum(diff*diff))


### Codercise H.8.3. 
Manually find values for  and  for which LCU outperforms Trotter, and vice-versa. Run the code below to test, then fill in some example values in the provided variables. Note that we set  and . The depth of the optimal Trotter circuit is provided as trotter_depth(alpha, error) and the depth of the optimal LCU circuit as LCU_depth(alpha, error).

In [ ]:
alpha, error = 2, 1e-2 # VARY THIS

print("For α =", alpha, "and error ε =", error, 
      "the optimal Trotter circuit has depth",  trotter_depth(alpha, error),
      "and the optimal LCU circuit depth", LCU_depth(alpha, error), ".")

alpha_trotter, error_trotter = 0, 1 # RECORD PARAMETERS FOR WHICH TROTTER IS BEST
alpha_LCU, error_LCU = 2, 0.01 # RECORD PARAMETERS FOR WHICH LCU IS BEST
